In [1]:
import numpy as np
import tensorflow as tf
sess = tf.InteractiveSession()
import melt

tensorflow_version: 0.12.head


In [2]:
logits = tf.constant([
                    [[15.2, 23.3, 31.2],
                    [29.3, 29.2, 2.2]],
                    [[15.2, 23.3, 31.2],
                    [29.3, 29.2, 2.2]]
                  ])
labels = tf.constant([[1, 0], [1, 2]])
mask = tf.ones_like(labels, dtype=tf.float32)

In [3]:
logits.eval()

array([[[ 15.19999981,  23.29999924,  31.20000076],
        [ 29.29999924,  29.20000076,   2.20000005]],

       [[ 15.19999981,  23.29999924,  31.20000076],
        [ 29.29999924,  29.20000076,   2.20000005]]], dtype=float32)

In [4]:
labels.eval()

array([[1, 0],
       [1, 2]], dtype=int32)

In [13]:
def exact_predict_loss(logits, targets, mask, num_steps, batch_size=None):
  """
  the same as using sparse_softmax_cross_entropy_with_logits 
  here mainly for debug, comparing experimenting purpose!
  logits  [batch_size, num_steps, vocab_size]
  targets [batch_size, num_steps]
  mast    [batch_size, num_steps]
  """
  if batch_size is None:
    batch_size = tf.shape(logits)[0]
  i = tf.constant(0, dtype=tf.int32)
  condition = lambda i, log_probs, log_probs_list: tf.less(i, num_steps)
  log_probs = tf.zeros([batch_size,], dtype=tf.float32)
  #---log_probs_list is for debug purpose, actually the whole function here is for deubg purpose
  log_probs_list = tf.TensorArray(
            dtype=tf.float32, tensor_array_name="log_probs_list", size=num_steps + 2, infer_shape=False)
            #dtype=tf.float32, tensor_array_name="log_probs_list", size=0, dynamic_size=True, infer_shape=False)
  def body(i, log_probs, log_probs_list):
    #@TODO can we not clac softmax for mask==0 ?
    step_logits = logits[:, i, :]
    step_probs = tf.nn.softmax(step_logits)
    step_targets = targets[:, i]
    selected_probs = melt.dynamic_gather2d(step_probs, step_targets)
    #TODO is this ok? or just use tf.nn.log_softmax to replace tf.nn.softmax
    selected_log_probs = tf.log(tf.maximum(selected_probs, 1e-12))
    step_mask = mask[:, i]
    masked_log_probs = selected_log_probs * step_mask
    log_probs_list = log_probs_list.write(i, masked_log_probs)
    log_probs += masked_log_probs
    return tf.add(i, 1), tf.reshape(log_probs, [batch_size,]), log_probs_list
  _, log_probs, log_probs_list = tf.while_loop(condition, body, [i, log_probs, log_probs_list])
    
  log_probs_list = log_probs_list.write(num_steps, log_probs)
  log_probs_list = log_probs_list.write(num_steps + 1, tf.exp(log_probs))
  log_probs_list = tf.transpose(log_probs_list.stack(), [1, 0])
  tf.add_to_collection('seq2seq_log_probs_list', log_probs_list)

  loss = -log_probs;
  return loss, log_probs_list


In [14]:
u, v = exact_predict_loss(logits, labels,mask, 2)

In [15]:
u.eval()

array([  8.54477024,  35.53139496], dtype=float32)

In [16]:
v.eval()

array([[ -7.90037251e+00,  -6.44397378e-01,  -8.54477024e+00,
          1.94559951e-04],
       [ -7.90037251e+00,  -2.76310215e+01,  -3.55313950e+01,
          3.70604972e-16]], dtype=float32)